In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
data = pd.read_csv('train_set_100.csv', encoding='utf-8')
data['mahkemes'] = data['mahkemes'].values.astype(str)
data['ictihats'] = data['ictihats'].values.astype(str)
for i,mahkeme in enumerate(data['mahkemes']):
    data['mahkemes'][i] = mahkeme.replace(" ", "_")
print(data.mahkemes.unique())
data['labels'] = pd.factorize(data.mahkemes)[0]

['undefined' 'asliye_hukuk_mahkemesi' 'other' 'ticaret_mahkemesi'
 'asliye_ceza_mahkemesi' 'ceza_dairesi' 'cocuk_mahkemesi']


In [2]:
#data.head()
#data.iloc[:, 1] = data.iloc[:, 1].apply(lambda x: '__label__' + x)

In [3]:
#data split for train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [4]:
train.head()

,ictihats,mahkemes,labels
0,taraf 6831 2b maddes orma beyan hanes şerh kay...,asliye_hukuk_mahkemesi,1
1,taraf çocuk kor istek ilişk başkan kurul 10012...,other,2
2,hük oe suç mahkumiyet çocuk nitelikli cinsel s...,other,2
3,hük mahkumiyet görüşülüp düşünül yüklene suç y...,asliye_ceza_mahkemesi,4
4,2709 141 6100 31 taraf görüle dav sonuç verile...,undefined,0


In [5]:
test.head()

,ictihats,mahkemes,labels
0,hük cinsel taciz suç mahkumiyet tehdit suç kar...,asliye_ceza_mahkemesi,4
1,hük beraat nitelikli resmi belge düzenlenme be...,other,2
2,tarih bölge adli mahkeme verile müddet temyize...,other,2
3,taraf tazminat mahkemeç sonuç kabul verile hük...,other,2
4,taraf muhakemes mahalli mahkemeç verile tarih ...,cocuk_mahkemesi,6


In [6]:
#Data prepare for fasttext format for train
train["label_format"]=0
train["label_format_without_ictihats"]=0
for i in range(len(train)):
    train.label_format[i]="__label__"+str(train.mahkemes[i])+" "+str(train.ictihats[i])
    train.label_format_without_ictihats[i]="__label__"+str(train.mahkemes[i])

C:\Users\onkol\AppData\Local\Temp/ipykernel_30168/3046627546.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["label_format"]=0
C:\Users\onkol\AppData\Local\Temp/ipykernel_30168/3046627546.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["label_format_without_ictihats"]=0
C:\Users\onkol\AppData\Local\Temp/ipykernel_30168/3046627546.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

In [7]:
#Data prepare for fasttext format for test
test["label_format"]=0
test["label_format_without_ictihats"]=0
for i in range(len(test)):
    test.label_format[i]="__label__"+str(test.mahkemes[i])+" "+str(test.ictihats[i])
    test.label_format_without_ictihats[i]="__label__"+str(test.mahkemes[i])

C:\Users\onkol\AppData\Local\Temp/ipykernel_30168/708011051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["label_format"]=0
C:\Users\onkol\AppData\Local\Temp/ipykernel_30168/708011051.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["label_format_without_ictihats"]=0
C:\Users\onkol\AppData\Local\Temp/ipykernel_30168/708011051.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

In [8]:
train.label_format.to_csv('fasttext_train.txt',index=None,header=None)
test.label_format.to_csv('fasttext_test.txt',index=None,header=None)


In [9]:
import fasttext
#train classifier model
model = fasttext.train_supervised('fasttext_train.txt',epoch=100,lr=0.05,label_prefix='__label__',dim=300)
# model.save_model('model.bin')


In [10]:
model.test('fasttext_test.txt')
# print(model_test)
# precision = model_test[1]
# recall = model_test[2]
# f1_score = (2*(precision*recall))/(precision + recall)
# result = model.test('fasttext_train.txt')
# validation = model.test('fasttext_test.txt')
# text_line =  "accuracy:" + str(result[1])  + "\nvalidation:" + str(validation[1]) + '\n' 
# print ('Precision:', precision)
# print ('F1 score:', f1_score)
# print ('Recall:', recall)
# print(text_line)



(5571, 0.8930174115957638, 0.8930174115957638)

In [11]:

y_pred = test.iloc[:, 0].apply(lambda x: model.predict(x)[0][0])
#print(y_pred.unique())
y_test = test.label_format_without_ictihats
#print(y_test.unique())

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

                                 precision    recall  f1-score   support

 __label__asliye_ceza_mahkemesi       0.90      0.93      0.91      1950
__label__asliye_hukuk_mahkemesi       0.82      0.81      0.81       243
          __label__ceza_dairesi       0.95      0.93      0.94       232
       __label__cocuk_mahkemesi       0.76      0.69      0.72        32
                 __label__other       0.87      0.85      0.86      1719
     __label__ticaret_mahkemesi       0.60      0.32      0.42        28
             __label__undefined       0.93      0.93      0.93      1367

                       accuracy                           0.89      5571
                      macro avg       0.83      0.78      0.80      5571
                   weighted avg       0.89      0.89      0.89      5571

[[1807    1    4    0  123    0   15]
 [   2  196    1    7   25    3    9]
 [   5    0  215    0    5    0    7]
 [   0    8    0   22    2    0    0]
 [ 187   10    3    0 1458    1   60]
 [  

In [12]:
model.predict('bicaklamak')


(('__label__undefined',), array([1.00001001]))